<a href="https://colab.research.google.com/github/KolossalKode/FastCap_CRM/blob/main/Strat%20Basic%20Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install alpha_vantage pandas openpyxl plotly kaleido PyPDF2
!pip install google-colab # Ensure google.colab is available if not in a standard Colab env

import json
try:
    from google.colab import userdata # Import userdata for Colab secrets
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False # Not running in Colab or userdata not available
    print("Warning: google.colab.userdata not found. Falling back to config file method for API key.")

from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from datetime import date, datetime, timedelta
import os
import numpy as np
import plotly.graph_objects as go
# from PyPDF2 import PdfMerger # No longer merging PDFs, saving HTML instead
import tempfile
import warnings
import time # Import time for sleep delays

# Suppress specific warnings if they become noisy (e.g., from pandas)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None # Suppress SettingWithCopyWarning


# ===============
#  CONFIGURATION
# ===============
# --- User Configuration ---
CONFIG_FILE_PATH = "config.json"
SYMBOLS = ['SPY', 'QQQ', 'XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV',
           'XLI', 'XLB', 'XLRE', 'XLK', 'XLU']
PERFORMANCE_LOOKAHEAD = 10
MIN_HIGHER_TFS_FOR_FTFC = 3
COLAB_SECRET_NAME = 'API_Key'
SLEEP_BETWEEN_SYMBOLS = 1 # seconds
# --- Charting Configuration ---
CREATE_CHARTS = True # Set to False to disable chart generation
CHART_OUTPUT_DIR = "charts" # Subdirectory to save HTML charts
# --- End User Configuration ---


# Alpha Vantage API function mapping and timeframe processing order
TIMEFRAMES_API = {
    'TIME_SERIES_INTRADAY': ['15min', '60min'],
    'TIME_SERIES_DAILY_ADJUSTED': 'Daily',
    'TIME_SERIES_WEEKLY_ADJUSTED': 'Weekly',
    'TIME_SERIES_MONTHLY_ADJUSTED': 'Monthly'
}
TIMEFRAME_ORDER = ['15min', '60min', 'Daily', 'Weekly', 'Monthly', 'Quarterly', 'Yearly']

# --- MODIFIED: Define Strat patterns as tuples for easier checking ---
REVERSAL_PATTERNS_STRAT = {
    # 3-Bar Patterns
    ("3", "1", "2u"): "3-1-2u",
    ("3", "1", "2d"): "3-1-2d",
    ("2u", "1", "2d"): "2u-1-2d",
    ("2d", "1", "2u"): "2d-1-2u",
    # 2-Bar Patterns (ensure they don't overlap with ends of 3-bar)
    ("2u", "2d"): "2u-2d",
    ("2d", "2u"): "2d-2u",
}

# --- Load API Key ---
API_KEY = None
if COLAB_ENV:
    try: API_KEY = userdata.get(COLAB_SECRET_NAME); print(f"OK: API Key from Colab '{COLAB_SECRET_NAME}'.")
    except Exception as e: print(f"ERROR: Colab secret '{COLAB_SECRET_NAME}' not found or error: {e}"); exit()
else:
    try:
        with open(CONFIG_FILE_PATH, 'r') as f: config = json.load(f)
        API_KEY = config.get('alpha_vantage', {}).get('api_key')
        if API_KEY: print(f"OK: API Key from {CONFIG_FILE_PATH}.")
        else: print(f"ERROR: API Key not found in {CONFIG_FILE_PATH}."); exit()
    except Exception as e: print(f"ERROR: Loading API Key from {CONFIG_FILE_PATH}: {e}"); exit()
if not API_KEY: print("ERROR: API_KEY not loaded."); exit()

# Initialize Alpha Vantage client
try: ts = TimeSeries(key=API_KEY, output_format='pandas'); print("OK: Alpha Vantage client initialized.")
except Exception as e: print(f"ERROR: Initializing Alpha Vantage client: {e}"); exit()


# ================
#  DATA FETCHING
# ================
def get_data(symbol, function, interval=None):
    """ Fetches and cleans time series data from Alpha Vantage. """
    max_retries = 3; retry_delay = 10
    for attempt in range(max_retries):
        try:
            tf_name = interval or function.split('TIME_SERIES_')[-1].replace('_ADJUSTED','')
            print(f"   Attempt {attempt+1}/{max_retries}: Fetching {tf_name} for {symbol} ({function})...")
            # API Call (omitted specific calls for brevity - same as previous version)
            if function == 'TIME_SERIES_INTRADAY': df, _ = ts.get_intraday(symbol=symbol, interval=interval, outputsize='full', extended_hours=False)
            elif function == 'TIME_SERIES_DAILY': df, _ = ts.get_daily(symbol=symbol, outputsize='full')
            elif function == 'TIME_SERIES_DAILY_ADJUSTED': df, _ = ts.get_daily_adjusted(symbol=symbol, outputsize='full')
            elif function == 'TIME_SERIES_WEEKLY': df, _ = ts.get_weekly(symbol=symbol)
            elif function == 'TIME_SERIES_WEEKLY_ADJUSTED': df, _ = ts.get_weekly_adjusted(symbol=symbol)
            elif function == 'TIME_SERIES_MONTHLY': df, _ = ts.get_monthly(symbol=symbol)
            elif function == 'TIME_SERIES_MONTHLY_ADJUSTED': df, _ = ts.get_monthly_adjusted(symbol=symbol)
            else: print(f"Warning: Unknown function '{function}'."); return None

            if df is not None and not df.empty:
                # --- Data Cleaning ---
                df.columns = [c.split('. ')[-1].replace(' ', '_') if '.' in c else c.replace(' ', '_') for c in df.columns]
                try: df.index = pd.to_datetime(df.index, errors='coerce')
                except Exception as e: print(f"   ERROR converting index: {e}."); return None
                df = df[pd.notna(df.index)] # Drop NaT indices
                if not isinstance(df.index, pd.DatetimeIndex) or df.index.empty: print(f"   WARNING: Invalid DatetimeIndex after conversion."); return None
                df = df.iloc[::-1] # Oldest first
                ohlcv_cols = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']
                for col in ohlcv_cols:
                   if col in df.columns:
                       if not pd.api.types.is_numeric_dtype(df[col]): df[col] = pd.to_numeric(df[col], errors='coerce')
                       if np.isinf(df[col]).any(): df[col] = df[col].replace([np.inf, -np.inf], np.nan)
                essential_ohlc = ['open', 'high', 'low', 'close']
                if 'ADJUSTED' in function and 'adjusted_close' in df.columns: essential_ohlc.append('adjusted_close')
                df.dropna(subset=essential_ohlc, inplace=True)
                if df.empty: return None
                print(f"   OK: Fetched/cleaned {len(df)} rows for {symbol} - {tf_name}.")
                return df
            else: time.sleep(1); return None # No data returned
        except ValueError as ve: # Handle API errors
             print(f"   ERROR (ValueError): {ve}")
             if "rate limit" in str(ve): time.sleep(retry_delay * (2**attempt)) # Backoff for rate limit
             elif "premium endpoint" in str(ve).lower() or "Invalid API call" in str(ve) or "invalid symbol" in str(ve).lower(): return None # Don't retry these
             elif attempt == max_retries - 1: return None # Max retries for other ValueErrors
             else: time.sleep(retry_delay * (2**attempt))
        except Exception as e: # Handle other errors
            print(f"   ERROR (Unexpected): {type(e).__name__} - {e}")
            if attempt == max_retries - 1: return None
            else: time.sleep(retry_delay * (2**attempt))
    print(f"   Failed fetch for {symbol} - {tf_name} after retries.")
    return None

# ============================
#  CANDLESTICK LABELING
# ============================
def label_candlesticks(df):
    """ Applies 'The Strat' labels (1, 2u, 2d, 3). Assumes sorted DatetimeIndex. """
    if df is None or df.empty: return df
    if not isinstance(df.index, pd.DatetimeIndex): df['label'] = 'N/A'; return df
    if not df.index.is_monotonic_increasing: df = df.sort_index()
    if not all(col in df.columns for col in ['high', 'low']): df['label'] = 'N/A'; return df
    if not pd.api.types.is_numeric_dtype(df['high']) or not pd.api.types.is_numeric_dtype(df['low']):
         df['high']=pd.to_numeric(df['high'], errors='coerce'); df['low']=pd.to_numeric(df['low'], errors='coerce'); df.dropna(subset=['high', 'low'], inplace=True)
         if df.empty: df['label'] = 'N/A'; return df
    prev_high = df['high'].shift(1); prev_low = df['low'].shift(1); curr_high = df['high']; curr_low = df['low']
    is_inside = (curr_high <= prev_high) & (curr_low >= prev_low); is_up = (curr_high > prev_high) & (curr_low >= prev_low)
    is_down = (curr_high <= prev_high) & (curr_low < prev_low); is_outside = (curr_high > prev_high) & (curr_low < prev_low)
    df['label'] = 'N/A'; df.loc[is_inside, 'label'] = '1'; df.loc[is_up, 'label'] = '2u'; df.loc[is_down, 'label'] = '2d'; df.loc[is_outside, 'label'] = '3'
    if not df.empty: df.iloc[0, df.columns.get_loc('label')] = 'N/A'
    return df

# ============================
#  HAMMER / SHOOTER DETECTION
# ============================
def add_hammer_shooter_columns(df):
    """ Adds boolean 'is_hammer'/'is_shooter' columns. """
    if df is None or df.empty: return df
    required_cols = ['open', 'high', 'low', 'close']
    if not all(col in df.columns for col in required_cols): df['is_hammer']=False; df['is_shooter']=False; return df
    for col in required_cols:
         if not pd.api.types.is_numeric_dtype(df[col]): df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(subset=required_cols, inplace=True)
    if df.empty: df['is_hammer']=False; df['is_shooter']=False; return df
    body_size = (df['close'] - df['open']).abs().replace(0, 0.00001); candle_range = (df['high'] - df['low']).replace(0, 0.00001)
    upper_wick = df['high'] - df[['open', 'close']].max(axis=1); lower_wick = df[['open', 'close']].min(axis=1) - df['low']
    body_midpoint = (df['close'] + df['open']) / 2; candle_lower_half_top = df['low'] + 0.5 * candle_range; candle_upper_half_bottom = df['high'] - 0.5 * candle_range
    is_hammer = ( (lower_wick >= 1.9 * body_size) & (upper_wick < body_size * 0.8) & (body_midpoint >= candle_upper_half_bottom) )
    is_shooter = ( (upper_wick >= 1.9 * body_size) & (lower_wick < body_size * 0.8) & (body_midpoint <= candle_lower_half_top) )
    df.loc[:, 'is_hammer'] = is_hammer; df.loc[:, 'is_shooter'] = is_shooter
    return df

# ==================
#  DATA RESAMPLING
# ==================
def resample_data(df, freq='QE'):
    """ Resamples OHLCV data to a lower frequency (e.g., Quarterly, Yearly). """
    if df is None or df.empty: return None
    if not isinstance(df.index, pd.DatetimeIndex): return None
    if not df.index.is_monotonic_increasing: df = df.sort_index()
    agg_dict = {'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'}
    if 'volume' in df.columns and pd.api.types.is_numeric_dtype(df['volume']): agg_dict['volume'] = 'sum'
    close_col_to_use = 'adjusted_close' if 'adjusted_close' in df.columns and pd.api.types.is_numeric_dtype(df['adjusted_close']) else 'close'
    agg_dict[close_col_to_use] = 'last'
    if close_col_to_use == 'adjusted_close' and 'close' in df.columns: agg_dict['close'] = 'last'
    if 'dividend_amount' in df.columns and pd.api.types.is_numeric_dtype(df['dividend_amount']): agg_dict['dividend_amount'] = 'sum'
    if 'split_coefficient' in df.columns and pd.api.types.is_numeric_dtype(df['split_coefficient']): agg_dict['split_coefficient'] = 'prod'
    try:
        df_resampled = df.resample(freq).agg(agg_dict)
        df_resampled.dropna(subset=['open', 'high', 'low', 'close'], how='all', inplace=True)
        if df_resampled.empty: return None
        if not isinstance(df_resampled.index, pd.DatetimeIndex): return None
        if close_col_to_use == 'adjusted_close' and 'adjusted_close' in df_resampled.columns:
             if 'close' in df_resampled.columns: df_resampled['close'] = df_resampled['adjusted_close']; df_resampled.drop(columns=['adjusted_close'], inplace=True, errors='ignore')
             else: df_resampled.rename(columns={'adjusted_close': 'close'}, inplace=True)
        df_resampled = label_candlesticks(df_resampled)
        df_resampled = add_hammer_shooter_columns(df_resampled)
        return df_resampled
    except Exception as e: print(f"Error during resampling to '{freq}': {e}"); return None

# ===================================================
#   HISTORICAL FTFC REVERSAL ANALYSIS
# ===================================================
def analyze_historical_ftfc_reversals(symbol_data, symbol, timeframe_order, lookahead=10, min_higher_tfs=3):
    """
    Analyzes historical data FOR A SINGLE SYMBOL to find FTFC reversals,
    identifies the Strat pattern, and calculates performance.
    """
    print(f"   Analyzing Symbol: {symbol} for FTFC Reversals...")
    historical_reversals = []

    # --- Precompute aligned labels ---
    aligned_labels_cache = {}
    available_tfs_info = []
    for tf in timeframe_order:
         df = symbol_data.get(tf)
         if df is not None and not df.empty and isinstance(df.index, pd.DatetimeIndex):
             if 'label' in df.columns: available_tfs_info.append({'name': tf, 'index': df.index, 'labels': df['label']})
    if not available_tfs_info: return pd.DataFrame()
    all_valid_indices = pd.Index([])
    if available_tfs_info:
        all_valid_indices = available_tfs_info[0]['index']
        for i in range(1, len(available_tfs_info)):
            try:
                if isinstance(all_valid_indices, pd.DatetimeIndex) and isinstance(available_tfs_info[i]['index'], pd.DatetimeIndex):
                     all_valid_indices = all_valid_indices.union(available_tfs_info[i]['index'])
            except Exception as e: print(f"      Error during index union for {symbol} at index {i}: {e}")
    if all_valid_indices.empty: return pd.DataFrame()
    if not isinstance(all_valid_indices, pd.DatetimeIndex):
         all_valid_indices = pd.to_datetime(all_valid_indices, errors='coerce').dropna()
         if not isinstance(all_valid_indices, pd.DatetimeIndex) or all_valid_indices.empty: return pd.DataFrame()
    base_aligned_df = pd.DataFrame(index=all_valid_indices.sort_values())
    for tf_info in available_tfs_info:
        tf_name = tf_info['name']; label_series = tf_info['labels']
        if label_series is not None and isinstance(label_series.index, pd.DatetimeIndex):
            if isinstance(base_aligned_df.index, pd.DatetimeIndex):
                try:
                    if not label_series.index.is_monotonic_increasing: label_series = label_series.sort_index()
                    temp_aligned = pd.merge_asof(
                        base_aligned_df, label_series.rename(f'label_{tf_name}'),
                        left_index=True, right_index=True, direction='backward', tolerance=pd.Timedelta('30 days')
                    )
                    aligned_labels_cache[tf_name] = temp_aligned[f'label_{tf_name}']
                except Exception as e: print(f"      ERROR during merge_asof for {symbol}-{tf_name}: {e}. Skipping.")

    # --- Iterate through timeframes to find reversals ---
    available_tfs_with_aligned_labels = sorted(
         [tf for tf in timeframe_order if tf in aligned_labels_cache], key=lambda x: timeframe_order.index(x)
    )
    if not available_tfs_with_aligned_labels: return pd.DataFrame()

    for i, smaller_tf in enumerate(available_tfs_with_aligned_labels):
        smaller_df = symbol_data.get(smaller_tf)
        if smaller_df is None or smaller_df.empty or not isinstance(smaller_df.index, pd.DatetimeIndex): continue

        higher_tfs = []
        for j in range(i + 1, len(available_tfs_with_aligned_labels)):
            higher_tfs.append(available_tfs_with_aligned_labels[j]);
            if len(higher_tfs) == min_higher_tfs: break
        if len(higher_tfs) < min_higher_tfs: continue

        try:
            aligned_smaller_labels_series = aligned_labels_cache[smaller_tf]
            aligned_higher_labels_series_list = [aligned_labels_cache[htf] for htf in higher_tfs]
            aligned_smaller_labels = aligned_smaller_labels_series.reindex(smaller_df.index)
            aligned_higher_labels_list = [s.reindex(smaller_df.index) for s in aligned_higher_labels_series_list]
        except Exception as e: print(f"      Error accessing/reindexing labels for {symbol} on {smaller_tf}: {e}"); continue

        try:
            labels_df = pd.DataFrame({'smaller_label': aligned_smaller_labels, **{f'higher_label_{k}': s for k, s in enumerate(aligned_higher_labels_list)}})
        except Exception as e: print(f"      Error creating labels_df for {symbol}-{smaller_tf}: {e}"); continue
        labels_df.dropna(inplace=True)
        if labels_df.empty: continue

        required_label_cols = ['smaller_label'] + [f'higher_label_{k}' for k in range(min_higher_tfs)]
        if not all(col in labels_df.columns for col in required_label_cols): continue

        htf_2u_trend = (labels_df[f'higher_label_0'] == '2u'); htf_2d_trend = (labels_df[f'higher_label_0'] == '2d')
        for k in range(1, min_higher_tfs): htf_2u_trend &= (labels_df[f'higher_label_{k}'] == '2u'); htf_2d_trend &= (labels_df[f'higher_label_{k}'] == '2d')
        smaller_tf_rev_vs_2u = labels_df['smaller_label'].isin(['2d', '1', '3']); smaller_tf_rev_vs_2d = labels_df['smaller_label'].isin(['2u', '1', '3'])
        reversal_bullish_setup = htf_2u_trend & smaller_tf_rev_vs_2u; reversal_bearish_setup = htf_2d_trend & smaller_tf_rev_vs_2d
        reversal_indices = labels_df.index[reversal_bullish_setup | reversal_bearish_setup]
        if reversal_indices.empty: continue

        print(f"         Found {len(reversal_indices)} potential FTFC reversal points for {symbol} on {smaller_tf}.")

        # --- Analyze Performance & Strat Pattern ---
        for reversal_time in reversal_indices:
            try:
                reversal_iloc = smaller_df.index.get_loc(reversal_time)
                if reversal_iloc + 1 + lookahead > len(smaller_df): continue

                trend = '2u' if htf_2u_trend.loc[reversal_time] else ('2d' if htf_2d_trend.loc[reversal_time] else 'Mixed')
                reversal_label = labels_df.loc[reversal_time, 'smaller_label'] # The single candle label causing FTFC break
                higher_tf_labels_at_reversal = {f'Higher TF{k+1} Label': labels_df.loc[reversal_time, f'higher_label_{k}'] for k in range(min_higher_tfs)}

                # --- Check for Strat Pattern ---
                strat_pattern_found = "N/A" # Default if no pattern matches
                # Check 3-bar pattern ending at reversal_iloc
                if reversal_iloc >= 2:
                    label_seq_3 = tuple(smaller_df['label'].iloc[reversal_iloc-2 : reversal_iloc+1])
                    strat_pattern_found = REVERSAL_PATTERNS_STRAT.get(label_seq_3, "N/A")

                # If no 3-bar found, check 2-bar pattern ending at reversal_iloc
                if strat_pattern_found == "N/A" and reversal_iloc >= 1:
                    label_seq_2 = tuple(smaller_df['label'].iloc[reversal_iloc-1 : reversal_iloc+1])
                    strat_pattern_found = REVERSAL_PATTERNS_STRAT.get(label_seq_2, "N/A")
                # --- End Strat Pattern Check ---

                reversal_candle = smaller_df.iloc[reversal_iloc]; future_candles = smaller_df.iloc[reversal_iloc + 1 : reversal_iloc + 1 + lookahead]
                if future_candles.empty: continue

                entry_price_col = 'adjusted_close' if 'adjusted_close' in reversal_candle.index and pd.notna(reversal_candle['adjusted_close']) else 'close'
                close_price_col = 'adjusted_close' if 'adjusted_close' in future_candles.columns else 'close'
                open_price_col = 'open'
                entry_price = reversal_candle[entry_price_col]
                if pd.isna(entry_price) or entry_price == 0: continue

                perf_data = {'Symbol': symbol, 'Reversal Time': reversal_time, 'Reversal Timeframe': smaller_tf,
                             'FTFC Trigger Label': reversal_label, # Renamed original label
                             'Strat Pattern': strat_pattern_found, # Added Strat pattern
                             'Higher TF Trend': trend, 'Entry Price': entry_price,
                             'Higher TFs Used': ", ".join(higher_tfs), **higher_tf_labels_at_reversal}

                for k in range(1, lookahead + 1):
                    actual_candle_index = k - 1; col_prefix = f'Fwd_{k}'
                    if actual_candle_index < len(future_candles):
                        candle = future_candles.iloc[actual_candle_index]
                        if close_price_col not in candle.index or open_price_col not in candle.index: continue
                        current_close = candle[close_price_col]; current_open = candle[open_price_col]
                        if pd.isna(current_close) or pd.isna(current_open) or pd.isna(entry_price): gross_move_from_entry, perc_move_from_entry, candle_gross_move, candle_perc_move = np.nan, np.nan, np.nan, np.nan
                        else:
                            gross_move_from_entry = current_close - entry_price; perc_move_from_entry = (gross_move_from_entry / entry_price) * 100 if entry_price != 0 else 0
                            candle_gross_move = current_close - current_open; candle_perc_move = (candle_gross_move / current_open) * 100 if current_open != 0 else 0
                        perf_data[f'{col_prefix}_Candle_Close'] = current_close; perf_data[f'{col_prefix}_GrossMoveFromEntry'] = gross_move_from_entry
                        perf_data[f'{col_prefix}_PercMoveFromEntry'] = perc_move_from_entry; perf_data[f'{col_prefix}_Candle_GrossMove'] = candle_gross_move
                        perf_data[f'{col_prefix}_Candle_PercMove'] = candle_perc_move
                    else: perf_data[f'{col_prefix}_Candle_Close']=np.nan; perf_data[f'{col_prefix}_GrossMoveFromEntry']=np.nan; perf_data[f'{col_prefix}_PercMoveFromEntry']=np.nan; perf_data[f'{col_prefix}_Candle_GrossMove']=np.nan; perf_data[f'{col_prefix}_Candle_PercMove']=np.nan
                historical_reversals.append(perf_data)
            except Exception as e: print(f"         Unexpected Error processing reversal at {reversal_time} for {symbol} on {smaller_tf}: {type(e).__name__} - {e}")

    if not historical_reversals: return pd.DataFrame()
    else:
        results_df = pd.DataFrame(historical_reversals)
        # --- Update column order for new 'Strat Pattern' ---
        id_cols = ['Symbol', 'Reversal Time', 'Reversal Timeframe', 'FTFC Trigger Label', 'Strat Pattern', 'Higher TF Trend', 'Entry Price', 'Higher TFs Used']
        label_cols = sorted([col for col in results_df.columns if 'Label' in col and 'FTFC Trigger' not in col]) # Exclude the trigger label here
        perf_cols = sorted([col for col in results_df.columns if col.startswith('Fwd_')], key=lambda x: (int(x.split('_')[1]), x))
        final_cols = [col for col in id_cols + label_cols + perf_cols if col in results_df.columns]
        if 'Reversal Time' in results_df.columns: results_df['Reversal Time'] = pd.to_datetime(results_df['Reversal Time'])
        return results_df[final_cols]


# ===================================================
#  HELPER: SAVE DATAFRAMES TO EXCEL (Optional Data Dump)
# ===================================================
def save_dataframes_to_excel(symbol, data_dict, date_prefix):
    """ Saves processed dataframes (OHLC, labels) to an Excel file. Optional. """
    excel_filename = f"{date_prefix}_{symbol}_SourceData.xlsx"
    try:
        with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
            for timeframe in timeframe_order:
                df_data = data_dict.get(timeframe)
                if df_data is not None and not df_data.empty:
                    df_copy = df_data.copy()
                    if pd.api.types.is_datetime64tz_dtype(df_copy.index): df_copy.index = df_copy.index.tz_localize(None)
                    df_copy.to_excel(writer, sheet_name=f"Data_{timeframe}")
        print(f"   Source data saved to: {excel_filename}")
    except Exception as e: print(f"   Error saving source data to Excel for {symbol}: {e}")


# ===================================================
#  HELPER: AGGREGATE PERFORMANCE RESULTS
# ===================================================
def aggregate_performance_results(historical_df):
    """ Aggregates the historical performance results, calculating average moves. """
    if historical_df is None or historical_df.empty: print("   No historical data to aggregate."); return pd.DataFrame()
    print("   Aggregating performance results...")
    gross_move_entry_cols=[col for col in historical_df.columns if col.startswith('Fwd_') and col.endswith('_GrossMoveFromEntry')]
    perc_move_entry_cols=[col for col in historical_df.columns if col.startswith('Fwd_') and col.endswith('_PercMoveFromEntry')]
    candle_gross_cols=[col for col in historical_df.columns if col.startswith('Fwd_') and col.endswith('_Candle_GrossMove')]
    candle_perc_cols=[col for col in historical_df.columns if col.startswith('Fwd_') and col.endswith('_Candle_PercMove')]
    all_perf_cols = gross_move_entry_cols + perc_move_entry_cols + candle_gross_cols + candle_perc_cols
    # --- Update grouping keys ---
    grouping_keys = ['Symbol', 'Reversal Timeframe', 'Higher TF Trend', 'Strat Pattern', 'FTFC Trigger Label']
    valid_grouping_keys = [key for key in grouping_keys if key in historical_df.columns]
    if not valid_grouping_keys: print("   Error: None grouping keys found."); return pd.DataFrame()
    if len(valid_grouping_keys) < len(grouping_keys): print(f"   Warning: Missing grouping keys: {set(grouping_keys)-set(valid_grouping_keys)}")
    try:
        grouped = historical_df.groupby(valid_grouping_keys)
        numeric_perf_cols = historical_df[all_perf_cols].select_dtypes(include=np.number).columns.tolist()
        if not numeric_perf_cols: print("   Error: No numeric performance columns found."); return pd.DataFrame()
        summary = grouped[numeric_perf_cols].mean()
        summary['Count'] = grouped.size()
    except Exception as e: print(f"   Error during aggregation mean/count: {e}"); return pd.DataFrame()
    summary.columns = [f'Avg_{col}' if col != 'Count' else col for col in summary.columns]
    print(f"   Aggregation complete. Summary has {len(summary)} rows.")
    return summary.reset_index()

# ===================================================
#  CHARTING FUNCTION (NEW)
# ===================================================
def build_and_save_chart(df, symbol, timeframe, output_dir):
    """
    Creates an interactive Plotly candlestick chart with Strat labels and saves as HTML.
    """
    if df is None or df.empty or not isinstance(df.index, pd.DatetimeIndex):
        print(f"      Skipping chart for {symbol}-{timeframe}: Invalid data.")
        return

    print(f"      Generating chart for {symbol}-{timeframe}...")
    try:
        fig = go.Figure()

        # 1. Candlestick Trace
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name='OHLC'
            )
        )

        # 2. Strat Labels Trace
        # Position labels slightly above high or below low
        label_y_positions = np.where(
            df['label'].isin(['2d']), df['low'] * 0.998, # Below low for 2d
            df['high'] * 1.002 # Above high for 1, 2u, 3
        )
        # Adjust y-position further if hammer/shooter exists to avoid overlap
        if 'is_hammer' in df.columns: label_y_positions = np.where(df['is_hammer'], label_y_positions * 1.002, label_y_positions)
        if 'is_shooter' in df.columns: label_y_positions = np.where(df['is_shooter'], label_y_positions * 1.002, label_y_positions)


        fig.add_trace(
            go.Scatter(
                x=df.index[df['label'] != 'N/A'], # Don't plot N/A labels
                y=label_y_positions[df['label'] != 'N/A'],
                text=df['label'][df['label'] != 'N/A'],
                mode='text',
                name='Strat Labels',
                textfont=dict(size=10),
                showlegend=False
            )
        )

        # 3. Hammer/Shooter Indicators (Optional)
        if 'is_hammer' in df.columns and df['is_hammer'].any():
            hammer_df = df[df['is_hammer']]
            fig.add_trace(go.Scatter(
                x=hammer_df.index, y=hammer_df['high'] * 1.004, # Slightly higher position
                mode='text', text='H', name='Hammer',
                textfont=dict(color='blue', size=12), showlegend=False
            ))
        if 'is_shooter' in df.columns and df['is_shooter'].any():
            shooter_df = df[df['is_shooter']]
            fig.add_trace(go.Scatter(
                x=shooter_df.index, y=shooter_df['high'] * 1.004, # Slightly higher position
                mode='text', text='S', name='Shooter',
                textfont=dict(color='red', size=12), showlegend=False
            ))

        # 4. Layout Configuration
        fig.update_layout(
            title=f"{symbol} - {timeframe} - Strat Labels",
            xaxis_title="Date",
            yaxis_title="Price",
            xaxis_rangeslider_visible=False, # Disable range slider for cleaner look
            hovermode="x unified", # Show hover info for all traces at once
            height=600 # Adjust height as needed
        )

        # 5. Save as HTML
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        filepath = os.path.join(output_dir, f"{symbol}_{timeframe}_Chart.html")
        fig.write_html(filepath)
        # print(f"      Chart saved to: {filepath}") # Reduce noise

    except Exception as e:
        print(f"      ERROR generating chart for {symbol}-{timeframe}: {e}")


# ======================
#  MAIN WORKFLOW
# ======================
def main():
    """
    Main execution function (Sequential Processing):
    Loops through each symbol:
    1. Fetches data.
    2. Resamples.
    3. Labels candles & adds formations.
    4. Generates Charts (New).
    5. Analyzes historical FTFC reversals.
    6. Collects results.
    After loop:
    7. Combines detailed results.
    8. Aggregates performance results.
    9. Saves combined detailed (Excel & JSON) and summary (Excel & JSON).
    """
    start_time = datetime.now()
    print(f"--- Starting Sequential Analysis at {start_time.strftime('%Y-%m-%d %H:%M:%S')} ---")

    all_historical_reversals_list = []
    processed_symbols = []

    # --- Loop through each symbol ---
    for symbol in SYMBOLS:
        print(f"\n--- Processing Symbol: {symbol} ---")
        symbol_data = {}
        symbol_fetch_successful = True

        # 1) Fetch Data
        print(f" Phase 1: Fetching Data for {symbol}")
        for function, intervals_or_name in TIMEFRAMES_API.items():
            # (Fetching logic - same as previous version, omitted for brevity)
            if isinstance(intervals_or_name, list): # Intraday
                for tf_interval in intervals_or_name:
                    time.sleep(0.5); df = get_data(symbol, function, interval=tf_interval)
                    if df is not None and not df.empty: symbol_data[tf_interval] = df
                    else: symbol_fetch_successful = False;
            else: # Daily, Weekly, Monthly
                tf_name = intervals_or_name
                time.sleep(0.5); df = get_data(symbol, function)
                if df is not None and not df.empty: symbol_data[tf_name] = df
                else: symbol_fetch_successful = False;

        if not symbol_data: print(f" Warning: No data fetched for {symbol}. Skipping."); continue
        required_base_tfs = [tf for tfs in TIMEFRAMES_API.values() for tf in (tfs if isinstance(tfs, list) else [tfs])]
        if not all(tf in symbol_data for tf in required_base_tfs):
             print(f" Warning: Missing required base timeframes for {symbol}. Skipping analysis.")
             time.sleep(SLEEP_BETWEEN_SYMBOLS); continue

        # 2) Resample
        print(f" Phase 2: Resampling Higher Timeframes for {symbol}")
        monthly_data = symbol_data.get('Monthly')
        if monthly_data is not None and not monthly_data.empty and isinstance(monthly_data.index, pd.DatetimeIndex):
            monthly_data_sorted = monthly_data.sort_index()
            quarterly_data = resample_data(monthly_data_sorted.copy(), 'QE')
            yearly_data = resample_data(monthly_data_sorted.copy(), 'YE')
            if quarterly_data is not None and not quarterly_data.empty: symbol_data['Quarterly'] = quarterly_data
            if yearly_data is not None and not yearly_data.empty: symbol_data['Yearly'] = yearly_data

        # 3) Label Candles & Add Formations
        print(f" Phase 3: Labeling Candles for {symbol}")
        valid_data_exists_after_labeling = False
        for timeframe, df_data in list(symbol_data.items()):
            if df_data is not None and not df_data.empty and isinstance(df_data.index, pd.DatetimeIndex):
                df_sorted = df_data.sort_index()
                labeled_df = label_candlesticks(df_sorted)
                if labeled_df is not None and 'label' in labeled_df.columns:
                     final_df = add_hammer_shooter_columns(labeled_df)
                     symbol_data[timeframe] = final_df # Update dict with processed data
                     valid_data_exists_after_labeling = True
                else: del symbol_data[timeframe] # Remove failed labeling
            elif df_data is not None: del symbol_data[timeframe] # Remove non-datetime index data

        if not valid_data_exists_after_labeling or not symbol_data:
             print(f" Warning: No valid labeled data remaining for {symbol}. Skipping analysis & charting.")
             time.sleep(SLEEP_BETWEEN_SYMBOLS); continue

        # 4) Generate Charts (New Step)
        if CREATE_CHARTS:
            print(f" Phase 4: Generating Charts for {symbol}")
            symbol_chart_dir = os.path.join(CHART_OUTPUT_DIR, symbol)
            for timeframe, df_chart_data in symbol_data.items():
                 build_and_save_chart(df_chart_data, symbol, timeframe, symbol_chart_dir)
        else:
             print(f" Phase 4: Skipping Chart Generation for {symbol}")


        # 5) Analyze Historical FTFC Reversals
        print(f" Phase 5: Analyzing FTFC Reversals for {symbol}")
        symbol_reversals_df = analyze_historical_ftfc_reversals(
            symbol_data, symbol, TIMEFRAME_ORDER,
            lookahead=PERFORMANCE_LOOKAHEAD, min_higher_tfs=MIN_HIGHER_TFS_FOR_FTFC
        )

        # 6) Collect Results
        if symbol_reversals_df is not None and not symbol_reversals_df.empty:
            all_historical_reversals_list.append(symbol_reversals_df)
            processed_symbols.append(symbol)
            print(f"   Finished analysis for {symbol}. Found {len(symbol_reversals_df)} reversals.")
        else:
            print(f"   No reversals found or analysis failed for {symbol}.")

        print(f"   Pausing for {SLEEP_BETWEEN_SYMBOLS} sec...")
        time.sleep(SLEEP_BETWEEN_SYMBOLS)
        # --- End of Symbol Loop ---


    # --- Post-Loop Processing ---
    print(f"\n--- Finished Processing All Symbols. Successfully processed: {processed_symbols} ---")

    # 7) Combine Detailed Results
    if not all_historical_reversals_list:
        print("No historical reversals found across any symbols.")
        combined_historical_reversals_df = pd.DataFrame()
    else:
        try:
            print("Combining detailed results...")
            combined_historical_reversals_df = pd.concat(all_historical_reversals_list, ignore_index=True)
            print(f"Combined results shape: {combined_historical_reversals_df.shape}")
            # Ensure 'Reversal Time' is datetime after concat
            if 'Reversal Time' in combined_historical_reversals_df.columns:
                 combined_historical_reversals_df['Reversal Time'] = pd.to_datetime(combined_historical_reversals_df['Reversal Time'])
        except Exception as e:
            print(f"Error combining historical reversal results: {e}")
            combined_historical_reversals_df = pd.DataFrame()

    # 8) Aggregate Combined Performance Results
    print("\n--- Aggregating Combined Performance Summary ---")
    performance_summary_df = aggregate_performance_results(combined_historical_reversals_df)


    # --- 9) Save Combined Results (Excel & JSON) ---
    today_str = date.today().strftime("%Y-%m-%d")

    # Save Detailed Results
    print("\n--- Saving Combined Detailed Performance Results ---")
    if not combined_historical_reversals_df.empty:
        detailed_excel_filename = f"{today_str}_FTFC_Reversal_Performance_Detailed_ALL.xlsx"
        detailed_json_filename = f"{today_str}_FTFC_Reversal_Performance_Detailed_ALL.json"
        try:
            df_to_save_detailed = combined_historical_reversals_df.copy()
            # Convert datetime to string for JSON compatibility (ISO format)
            if 'Reversal Time' in df_to_save_detailed.columns:
                 df_to_save_detailed['Reversal Time_str'] = df_to_save_detailed['Reversal Time'].dt.strftime('%Y-%m-%dT%H:%M:%S') # Create string version for JSON
                 # Make timezone naive for Excel
                 if pd.api.types.is_datetime64tz_dtype(df_to_save_detailed['Reversal Time']):
                      df_to_save_detailed['Reversal Time'] = df_to_save_detailed['Reversal Time'].dt.tz_localize(None)

            # Save Excel
            df_to_save_detailed.drop(columns=['Reversal Time_str'], errors='ignore').to_excel(detailed_excel_filename, index=False, engine='openpyxl')
            print(f"   Combined detailed results saved to: {detailed_excel_filename}")
            # Save JSON
            df_to_save_detailed.drop(columns=['Reversal Time'], errors='ignore').rename(columns={'Reversal Time_str':'Reversal Time'}).to_json(detailed_json_filename, orient='records', indent=4)
            print(f"   Combined detailed results saved to: {detailed_json_filename}")

        except Exception as e:
            print(f"   Error saving combined detailed performance results: {e}")
    else:
        print("   No combined historical reversals found, skipping detailed results save.")

    # Save Summary Results
    print("\n--- Saving Combined Performance Summary ---")
    if performance_summary_df is not None and not performance_summary_df.empty:
        summary_excel_filename = f"{today_str}_FTFC_Reversal_Performance_Summary_ALL.xlsx"
        summary_json_filename = f"{today_str}_FTFC_Reversal_Performance_Summary_ALL.json"
        try:
            # Save Excel
            performance_summary_df.to_excel(summary_excel_filename, index=False, engine='openpyxl')
            print(f"   Combined aggregated performance summary saved to: {summary_excel_filename}")
            # Save JSON
            performance_summary_df.to_json(summary_json_filename, orient='records', indent=4)
            print(f"   Combined aggregated performance summary saved to: {summary_json_filename}")
        except Exception as e:
            print(f"   Error saving combined performance summary: {e}")
    else:
        print("   No performance summary generated, skipping summary results save.")


    end_time = datetime.now()
    print(f"\n--- Analysis Complete at {end_time.strftime('%Y-%m-%d %H:%M:%S')} ---")
    print(f"Total execution time: {end_time - start_time}")


# ======================
#  SCRIPT EXECUTION
# ======================
if __name__ == "__main__":
    main()

OK: API Key from Colab 'API_Key'.
OK: Alpha Vantage client initialized.
--- Starting Sequential Analysis at 2025-07-11 12:52:18 ---

--- Processing Symbol: SPY ---
 Phase 1: Fetching Data for SPY
   Attempt 1/3: Fetching 15min for SPY (TIME_SERIES_INTRADAY)...
   OK: Fetched/cleaned 1288 rows for SPY - 15min.
   Attempt 1/3: Fetching 60min for SPY (TIME_SERIES_INTRADAY)...
   OK: Fetched/cleaned 337 rows for SPY - 60min.
   Attempt 1/3: Fetching DAILY for SPY (TIME_SERIES_DAILY_ADJUSTED)...
   OK: Fetched/cleaned 6461 rows for SPY - DAILY.
   Attempt 1/3: Fetching WEEKLY for SPY (TIME_SERIES_WEEKLY_ADJUSTED)...
   OK: Fetched/cleaned 1340 rows for SPY - WEEKLY.
   Attempt 1/3: Fetching MONTHLY for SPY (TIME_SERIES_MONTHLY_ADJUSTED)...
   OK: Fetched/cleaned 308 rows for SPY - MONTHLY.
 Phase 2: Resampling Higher Timeframes for SPY
 Phase 3: Labeling Candles for SPY
 Phase 4: Generating Charts for SPY
      Generating chart for SPY-15min...
      Generating chart for SPY-60min...
      

/tmp/ipython-input-1-338636654.py:639: DeprecationWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.



   Combined detailed results saved to: 2025-07-11_FTFC_Reversal_Performance_Detailed_ALL.xlsx
   Combined detailed results saved to: 2025-07-11_FTFC_Reversal_Performance_Detailed_ALL.json

--- Saving Combined Performance Summary ---
   Combined aggregated performance summary saved to: 2025-07-11_FTFC_Reversal_Performance_Summary_ALL.xlsx
   Combined aggregated performance summary saved to: 2025-07-11_FTFC_Reversal_Performance_Summary_ALL.json

--- Analysis Complete at 2025-07-11 12:54:37 ---
Total execution time: 0:02:18.700105
